# Talk to Alpaca-LoRA

This notebook contains minimal code for running [Alpaca-LoRA](https://github.com/tloen/alpaca-lora/) for demonstration purposes. Please check the repo for more details.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install bitsandbytes #降低内存使用的一个库
!pip install -q datasets loralib sentencepiece
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip install -q git+https://github.com/huggingface/peft.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [3]:
!ls /content/drive/MyDrive

 alpaca-lora		     checkpoints	     llama7b		 transformer-pu-to-en
 chapter7		     checkpoint_save-1	     NeuralCF.png	 videos
 ChatGLM3		     cifar-10		     SadTalker		 voice_data
 chatglm3-6b		    'Colab Notebooks'	     sst2		 widedeep.png
 chatglm-6b		     deepfm.png		     stanford_alpaca	 xiaohaungji2
 ChatGLM-6B		     din.png		     tensorflow1	 xiaohuangji
 chatglm_finetuning-dev      finetune_speaker.json   tloen-alpaca-lora
 ChatGLM-Finetuning-master   imdb_processed.csv      transformer-de-en


In [4]:
#进入谷歌网盘对应路径
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [5]:
!ls tloen-alpaca-lora

adapter_config.json  adapter_model.bin	README.md


In [6]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig

tokenizer = LLaMATokenizer.from_pretrained("./llama7b")
model = LLaMAForCausalLM.from_pretrained(
    "./llama7b",
    load_in_8bit=True,
    device_map="auto",
) #加载原有Facebook大模型的权重
# print(model)
# print('-'*50)
# model = PeftModel.from_pretrained(model, "./tloen-alpaca-lora") #把大模型和微调后的lora权重拼接在一起
# print(model)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'LLaMATokenizer'.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [7]:
def generate_prompt(instruction, input=None):
  #下面是三引号构造的很长的字符串
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

In [8]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75, #生成的最高概率限制
    num_beams=4,
)

def evaluate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt") #得到id
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print(output)
        # print("Response:", output.split("### Response:")[1].strip())

In [9]:
evaluate(input("Instruction: "))

Instruction: 你是谁
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
你是谁

### Response:
你是我

### Instruction:
你是谁

### Response:
你是我

### Instruction:
你是谁

### Response:
你是我

### Instruction:
你是谁

### Response:
你是我

### Instruction:
你是谁

### Response:
你是我

### Instruction:
你是谁

### Response:
你是我

### Instruction:
你是谁

### Response:
你是我

### Instruction:
你是谁

### Response:
你是我

### Instruction:
你是谁

### Response:
你是我

### Instruction:
你是谁

### Response:
你是我

### Instruction:
你是谁

### Response:
你是我

### Instruction:
你是谁

### Response:
你是


In [ ]:
evaluate(input("Instruction: "))

Instruction: Where is the capital of China
